In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from mpl_toolkits.mplot3d import Axes3D

In [2]:
#Define constants
mass = 39.95*1.68*10**(-27)
epsilon = 119.8/1.381*10**(-23)
sigma = 3.405 #angstrom
Electric_Charge = 0.0
Epsilon_0 = 8.85*10**(-12)*sigma*epsilon*Electric_Charge**2
N_Particles = 108

#User input
rho = 1.3 #units of sigma**-3, N*m/L^3
Temperature = 0.5
Box_Size = (N_Particles/rho)**(1/3)
Lattice_Size = Box_Size/3
#Box_Size = 100
#Lattice_Size = 3.12354
Particle_Charge = -1*Electric_Charge
E_Field = 0.2

In [3]:
#Functions

def Distance(A_Position, B_Position):

    Sequence_Polarity = np.array(A_Position>B_Position)*2 -1
    Image_Polarity = np.array(np.power((A_Position-B_Position),2)>(Box_Size/2)**(2))
    Distances = (Image_Polarity*Box_Size-np.abs(A_Position-B_Position))
    Distance = np.sum(np.power(Distances,2))**(0.5)
    Directions = Distances*Sequence_Polarity

    return Distance, Directions

def Kinetic_Energy(Velocity):
    Kin_En = np.sum(0.5* Velocity**2)
    return Kin_En


def Interaction(A_Position,B_Position):

    Dist_Norm, Dist_Vec = Distance(A_Position, B_Position)
    Lennard_Jones_pot = 4*(Dist_Norm**(-6) - Dist_Norm**(-12))
    Total_Force = 4*(6/(Dist_Norm)**(7) - 12/(Dist_Norm)**(13))
    
    Forces = (Dist_Vec/Dist_Norm)*Total_Force
    Forces[2] += Particle_Charge*E_Field
    if np.isclose(Dist_Norm, 0) == True:
        print('Overlapping')
    return Forces, Lennard_Jones_pot, Dist_Norm, Total_Force

#Maybe write to file multiple times and get values from this file to get <n(r)>
#Or run code again and use Distance_List1, Distance_List2 etc
Distance_List = []
def Dist_Pair_Corr(Dist_List, Distance):
    
    Dist_List.append(Distance)



#Calculated for each [r, r+delta_r]
#Distance here is every value of Pair_Corr_Bins





Pressure_LJ_Pot_list = []
def Average_LJ_Pot(Distance, Total_Force):
    Pressure_LJ_Pot = Distance*Total_Force
    Pressure_LJ_Pot_list.append(Pressure_LJ_Pot)
    return Pressure_LJ_Pot_list



def Particles_Initialization(rho, Temperature, N_Particles):
    
    box_size = (N_Particles/rho)**(1/3)

    pos = np.zeros((N_Particles,6))


    Step = 0

    '''Testing for just 4 particles'''
#     pos[:,0:3] = np.array([[0,0,0] ,[0,box_size/2,box_size/2],
#                           [box_size/2,0,box_size/2],
#                           [box_size/2,box_size/2,0]] )
    
#     for p in range(N_Particles):
#         for o in range(3):
#             pos[p,o+3] = np.random.normal(0, np.sqrt(Temperature))
    
    '''The following part is the correct one, for the 108 particles simulation '''
    l=-1 # Loop counter
    for i in range(3):
        for j in range(3):
            for k in range(3):
                l+=1
                # particle nr. 1
                pos[l,0]=0+box_size/3*i; pos[l,1]=0+box_size/3*j; pos[l,2]=0+box_size/3*k; # corner
                # particle nr. 2
                pos[l+27, 0]=box_size/6+box_size/3*i; pos[l+27, 1]=box_size/6+box_size/3*j; pos[l+27, 2]=0+box_size/3*k; # x-y plane
                # particle nr. 3
                pos[l+54,0]=box_size/6+box_size/3*i; pos[l+54,1]=0+box_size/3*j; pos[l+54,2]=box_size/6+box_size/3*k; # x-z plane
                # particle nr. 4
                pos[l+81,0]=0+box_size/3*i; pos[l+81,1]=box_size/6+box_size/3*j; pos[l+81,2]=box_size/6+box_size/3*k; # y-z plane


        
                
                Velocities = np.zeros((4,3))
                for m in range(4):
                    vx = np.random.normal(0, np.sqrt(Temperature))
                    vy = np.random.normal(0, np.sqrt(Temperature))
                    vz = np.random.normal(0, np.sqrt(Temperature))
                    Velocities[m,:] = np.array([vx,vy,vz])


                pos[Step:Step+4,3:] = Velocities
                Step += int(4)
    return pos

#Particles relaxation





def Integration_eq_motion(Particles, Iterations, Timestep, Rescaling):
    
    Dimensions = 3

    Historic_Positions = np.zeros((N_Particles,Dimensions,Iterations))
    Historic_Velocities = np.zeros((N_Particles,Dimensions,Iterations))
    
    Positions = Particles[:,:3]
    Velocities = Particles[:,3:]
    
    Historic_Positions[:,:,0] = Positions
    Historic_Velocities[:,:,0] = Velocities
    
    if Rescaling == True:
        #Fiducial extreme values of lambda. Inside these we assume the system is in thermal equilibrium
        lambda_min = 0.9
        lambda_max = 1.1
        Kin_Energy = np.zeros((Iterations))     
        for t in range(Iterations-1):
            Kin_Energy[t] = Kinetic_Energy(Velocities[:,:])
            Forces = np.zeros((N_Particles,Dimensions))
            for i in range(N_Particles):
                for j in range(N_Particles):
                    if (i != j):
                        Added_Forces, LJ_En, Distance, Total_Force = Interaction(Positions[i,:],Positions[j,:])
                        Forces[i] += Added_Forces          
            Positions = (Historic_Positions[:,:,t] + Velocities*Timestep + Forces*(Timestep**2)/2)%Box_Size
            Historic_Positions[:,:,t+1] = Positions
            if (t>0):
                Velocities = Historic_Velocities[:,:,t-1] + (Historic_Forces+Forces)*(Timestep/2)
                Historic_Velocities[:,:,t] = Velocities
            Historic_Forces = Forces
            lmbd = np.sqrt((N_Particles -1)*3*Temperature/Kin_Energy[t])         
            if lmbd < lambda_min or lmbd > lambda_max:
                Historic_Velocities[:,:,t] = Velocities * lmbd
            if (t%100) == 0 and t>0:
                print(t)
                print('lambda = ', lmbd)
        return Historic_Velocities[:,:,t], Historic_Positions[:,:,t]
    
    if Rescaling == False:
        
        Distance_List = []
        Pressure_LJ_Pot_list = []
        Energy = np.zeros((Iterations,3))
        
        for t in range(Iterations-1):
            Energy[t,0] = Kinetic_Energy(Velocities[:,:])

            Forces = np.zeros((N_Particles,Dimensions))
            Pot_En = np.zeros((N_Particles,Dimensions))
            for i in range(N_Particles):
                for j in range(N_Particles):
                    if (i != j):
                        Added_Forces, LJ_En, Particles_Distance, Total_Force = Interaction(Positions[i,:],Positions[j,:])
                        Forces[i] += Added_Forces
                        Pot_En[i] += LJ_En
                        if i < j and t in range(Iterations-20, Iterations-5):
                            Pressure_Value = Average_LJ_Pot(Particles_Distance, Total_Force)
                            if Particles_Distance < Box_Size/2:
                                Dist_Pair_Corr(Distance_List, Particles_Distance)

            Energy[t,1] = -np.sum(Pot_En)
            Energy[t,2] = Energy[t,0] + Energy[t,1]

            Positions = (Historic_Positions[:,:,t] + Velocities*Timestep + Forces*(Timestep**2)/2)%Box_Size
            Historic_Positions[:,:,t+1] = Positions
            if (t>0):
                Velocities = Historic_Velocities[:,:,t-1] + (Historic_Forces+Forces)*(Timestep/2)
                Historic_Velocities[:,:,t] = Velocities
            Historic_Forces = Forces
            if (t%100) == 0 and t>0:
                print(t)
        return Historic_Velocities, Historic_Positions, Energy, Pressure_Value, Distance_List

    
def Relaxation(Iterations, Particles, Timestep):

    Relaxed_Particles_Vel, Relaxed_Particles_Pos = Integration_eq_motion(Particles, Iterations, Timestep, Rescaling = True)
    
            
    return Relaxed_Particles_Vel, Relaxed_Particles_Pos

def Simulation(Iterations, Particles, Timestep):
    
    Velocities, Positions, Energy, Pressure_Value, Distance_Pair_Corr = Integration_eq_motion(Particles, Iterations, Timestep, Rescaling = False)
    return Velocities, Positions, Energy, Pressure_Value, Distance_Pair_Corr
    


def Pair_Corr_function(Average_n, Distance_r, Delta_r):
    Pair_Corr_r = 2*(Lattice_Size*3)**3/((N_Particles*(N_Particles-1)))*(Average_n/15)/(4*np.pi*Distance_r**2*Delta_r) #15 = normalization factor
    Pair_Corr_r_List.append(Pair_Corr_r)
    return Pair_Corr_r_List

In [4]:
Initial_Particles = Particles_Initialization(rho, Temperature, N_Particles)

In [ ]:
Iterations = 500
Timestep = 0.001
Rescaled_Velocities, Rescaled_Positions = Relaxation(Iterations, Initial_Particles, Timestep)

Rescaled_Particles = np.zeros((N_Particles, 6))

Rescaled_Particles[:,:3] = Rescaled_Positions
Rescaled_Particles[:,3:] = Rescaled_Velocities


In [ ]:
Iterations = 1000

Timestep = 0.001

Velocities, Positions, Energy, Pressure_Value, Distance_Pair_Corr = Simulation(Iterations, Rescaled_Particles, Timestep)




In [ ]:
start_plot = 0

end_plot = -1

plt.plot( Energy[start_plot:end_plot,0]/N_Particles, label = 'Kinetic energy')
plt.plot( Energy[start_plot:end_plot,1]/N_Particles, label = 'Potential energy')
plt.plot( Energy[start_plot:end_plot,2]/N_Particles, label = 'Total energy')
# plt.yscale('log')
plt.legend()
# plt.savefig('Energy_small_timestep_4_particles.png')

In [ ]:
for m in range(N_Particles):
    plt.scatter(Positions[m,0,start_plot:end_plot], Positions[m,1,start_plot:end_plot])
plt.xlim(0,Box_Size)
plt.ylim(0,Box_Size)

In [ ]:
Delta_r = Box_Size/100
Pair_Corr_Bins = np.arange(0, Box_Size/2, Delta_r)
print(len(Pair_Corr_Bins))
Pair_Corr_r_List = []

Avg_n = plt.hist(Distance_Pair_Corr, bins = Pair_Corr_Bins)
hist, bins = np.histogram(Distance_List, bins= Pair_Corr_Bins)
plt.xlabel('r (L)')
plt.ylabel('<n(r)>')
plt.title(r'Average distance for $\rho$ = {}'.format(rho) + r' and $T$ = {}'.format(Temperature))
plt.show()


In [ ]:
print(len(Pair_Corr_Bins))
Pair_Corr_r_List = []

print(type(hist))
print(type(bins))


for Average_n, Distance_r in zip(Avg_n[0], Pair_Corr_Bins[1:]):
    Pair_Corr_r_List =  Pair_Corr_function(Average_n, Distance_r, Delta_r)

print(len(Pair_Corr_r_List))

print(len(Pair_Corr_Bins))

plt.plot(Pair_Corr_Bins[1:], Pair_Corr_r_List)
plt.xlabel('r (L)')
plt.ylabel('g(r)')
# plt.xlim()
plt.title(r'Correlation function for $\rho$ = {}'.format(rho) + r' and $T$ = {}'.format(Temperature))
plt.show()


In [ ]:
#Pressure
#To get average over multiple initial conditions, save the Pressure_Value_Total ssomewhere
# and run the simulation multiple times
Pressure_Value_Total = np.mean(Pressure_Value)
print(np.sum(Pressure_Value))
print(len(Pressure_Value))
print(Pressure_Value_Total)
Pressure = Temperature*rho - (rho*(0.5*(Pressure_Value_Total)))/((3*N_Particles))
print(Pressure)

In [ ]:
# %matplotlib notebook

In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for l in range(N_Particles):
#     ax.scatter(Historic_Positions[l,0,0:],Historic_Positions[l,1,0:], Historic_Positions[l,2,0:])

# ax.view_init(elev=8., azim=60.)

# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')

# plt.show()

In [ ]:
print(np.tile([Lattice_Size/2,Lattice_Size/2,Lattice_Size/2],4).reshape(4,3))

In [ ]:
Lattice_Size = 3

Unit_Cell = np.array([[0,0,0],
                       [0,Lattice_Size,Lattice_Size],
                       [Lattice_Size,0,Lattice_Size],
                       [Lattice_Size,Lattice_Size,0]])

print(int(np.size(Unit_Cell)/3))